In [11]:
import cv2
import imutils
import dlib
import scipy
import numpy as np

from scipy.spatial import distance
from imutils import face_utils

import pygame

In [12]:
def light_remover(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab =  cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    L = lab[:,:,0]
    composed = cv2.cvtColor(lab,cv2.COLOR_LAB2BGR)
    # invert the image lighting
    Medianblur = cv2.medianBlur(L,99)
    invertedL = cv2.bitwise_not(Medianblur)
    composed = cv2.addWeighted(gray,0.80,invertedL,0.20,0)
    return composed

In [13]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
# 	print(ear)
	return ear

In [14]:
def mouth_aspect_ratio(muh):
    A = distance.euclidean(muh[13], muh[19])
    B = distance.euclidean(muh[15], muh[17])
    C = distance.euclidean(muh[12], muh[16])
    ear1 = (A + B) / (2.0 * C)
#     print(ear1)
    return ear1

In [15]:
def select_alarm() :
    sound_alarm("nomal_alarm.wav")


def sound_alarm(path) :
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()

In [16]:
threshEye = 0.32
threshLip = 0.28
frame_check = 60
# frame_check_eyes = 90
# frame_check_lip = 30
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(uStart, uEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

## Via Mobile

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read() # Our operations on the frame come here
    final  = light_remover(frame) # Display the resulting frame
    cv2.imshow('frame',final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyWindow('frame')

In [ ]:
url = 'http://192.168.43.1:8080/video'
cap = cv2.VideoCapture(url)
flag=0
global lipear
global ear
while True:
    ret, frame=cap.read()
    final  = light_remover(frame)
    frame = imutils.resize(final, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  #  gray = light_removing(frame)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips = shape[uStart:uEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        lipear = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        lipHull = cv2.convexHull(lips)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [lipHull], -1, (0, 255, 0), 1)
        if ear < threshEye or lipear > threshLip:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                if lipear > threshLip and ear < threshEye:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif ear < threshEye: 
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif lipear > threshLip:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        else:
            flag = 0
    final  = light_remover(frame)
    cv2.imshow("Frame", final)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break


# Via Laptop

In [18]:

cap = cv2.VideoCapture(0)
flag=0
global lipear
global ear
while True:
    ret, frame=cap.read()
    frame  = light_remover(frame)
    frame = imutils.resize(frame, width=450)
    subjects = detect(frame, 0)
    for subject in subjects:
        shape = predict(frame, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips = shape[uStart:uEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        lipear = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        lipHull = cv2.convexHull(lips)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [lipHull], -1, (0, 255, 0), 1)
        print(lipear)
        if ear < threshEye or lipear > threshLip:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                if lipear > threshLip and ear < threshEye:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif ear < threshEye: 
                    select_alarm()
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif lipear > threshLip:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break


0.02800417834795944
0.02800417834795944
0.02870810720377016
0.023199420949679552
0.023749016662636548
0.023782574707724703
0.023782574707724703
0.03287979746107146
0.044396937564865596
0.02870810720377016
0.02323069967623409
0.011624763874381928
0.02800417834795944
0.023199420949679552
0.023782574707724703
0.02870810720377016
0.0280419351108903
0.023749016662636548
0.033586181457325226
0.0280419351108903
0.011615349838117045
0.011624763874381928
0.016793090728662613
0.01642658527277326
0.023782574707724703
0.05583195306044417
0.06090319214176199
0.012180638428352398
0.017226024063738864
0.03285317054554652
0.0
0.011615349838117045
0.023782574707724703
0.0280419351108903
0.02800417834795944
0.06270367312858884
0.011615349838117045
0.06143827442103804
0.011615349838117045
0.011360702676647077
0.011624763874381928
0.03761852611988374
0.01642658527277326
0.011624763874381928
0.022703830459324992
0.016793090728662613
0.1316023560000482
0.02870810720377016
0.03363363969981562
0.0232306996762